# Vicuna 圧縮テスト
Vicuna_13bを無理やり圧縮して使う

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### データセット読み込み


In [2]:
dataset = load_dataset("kunishou/J-ResearchCorpus", split='train')
print(dataset[0])

{'filename': 'A1-1.pdf', 'text': '# ニューラル機械翻訳における Iterative Back-Translation を利用した コンパラブルコーパスの活用\n\n山本 優紀 秋葉 友良 塚田 元\n豊橋技術科学大学\n\\{yamamoto.yuki.pr, akiba.tomoyoshi.tk, tsukada.hajime.hl\\}@tut.jp\n\n## 概要\n\nニューラル機械翻訳 (NMT) の学習に用いる対訳コーパスの構築法として, 文書単位で対応付けられた 2 つの言語のコーパス (コンパラブルコーパス) から、対応付けられる文ペアを自動的に抽出する手法が広く採用されている. しかし, 文単位で意味が対応するものは少なく,多くの文は抽出されず捨てられてしまう. 本研究では、対訳コーパスとして抽出されなかった文を含めて,コンパラブルコー パス全体を NMT の学習に活用する手法を提案する. 評価実験により, コンパラブルコーパスでデータ拡張を行うことや, コンパラブル性の利用, Iterative Back-Translation の活用によって翻訳モデルの性能が向上することを確認した.\n\n## 1 はじめに\n\n機械翻訳の分野では, 深層学習の発達により, ニューラルネットワークを用いるニューラル機械翻訳 (Neural Machine Translation:NMT) が, 従来手法の統計的機械翻訳よりも高い性能を示しており, 様々な研究が行われている. NMT では, ニューラルネットワークで構築した翻訳モデルを, 翻訳元の言語 (原言語) の文と,その訳の言語 (目的言語) の文のぺアにした対訳コーパスを用いて学習を行う. NMT は, 対訳コーパスから翻訳に関わる様々な知識を学習するため, 対訳コーパスの質や量が NMT モデルの翻訳性能に大きく影響する.しかし, 大規模な対訳コーパスを人手で作成することは困難という問題点がある.\n\nこの問題の解決策として, 既存の日本語と英語の翻訳テキストから対訳コーパスを構築する手法が提案されている.[1]これは, 新聞などの文書単位で対応付けつけられた 2 つの言語コーパス (コンパラブルコーパス) から, 対応付けられる文ぺアを自動的\nに抽出するこ

### モデル読み込み

In [3]:
model_name = "helloollel/vicuna-7b"
model_id = "lmsys/vicuna-13b-v1.5"

In [4]:
from torch import bfloat16
import transformers

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

In [5]:
# Load model directly


# model = AutoModelForCausalLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# vicuna 13b Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# vicuna 13b Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()

tokenizer_config.json: 100%|██████████| 749/749 [00:00<00:00, 2.10MB/s]
tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 26.6MB/s]
config.json: 100%|██████████| 638/638 [00:00<00:00, 2.43MB/s]
pytorch_model.bin.index.json: 100%|██████████| 33.4k/33.4k [00:00<00:00, 29.3MB/s]
pytorch_model-00001-of-00003.bin:  13%|█▎        | 1.34G/9.95G [00:12<01:20, 107MB/s]


KeyboardInterrupt: 

### モデルのテスト
今回は要約タスクをさせます．

In [ ]:
def summarize_text(text):
    inputs = tokenizer.encode("あなたは教師です．以下の論文の内容を解釈し，生徒に説明してください．: " + text, return_tensors="pt", max_length=512, truncation=True)
    print(f"inputs:{inputs}")
    outputs = model.generate(inputs, max_new_tokens=100, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [ ]:
sample_text = dataset[0]['text']
summary = summarize_text(sample_text)


inputs:tensor([[    1, 29871, 30641, 30371, 30366, 30449, 31023, 31549, 30499, 30427,
           242,   191,   145, 30651, 30557, 30199, 31871, 30333, 30199, 30728,
         31294, 30396, 31201,   236,   138,   139, 30326, 30214, 30486,   232,
           193,   149, 30353,   235,   173,   175, 30592, 30326, 30466, 30568,
         30955, 30566, 30298,   242,   191,   145, 29901,   396, 29871, 30635,
         30645, 30185, 30281, 30258, 31540,   233,   165,   179,   234,   194,
           190,   235,   171,   182, 30353, 30697, 30807, 30332, 20504,  1230,
          7437, 29899,  4300, 18411, 29871, 30396, 31107, 30406, 30326, 30366,
         29871, 30459, 30203, 30715, 30281, 30582, 30258, 30459, 30185, 30715,
         30255, 30199, 31704, 30406,    13,    13, 30329, 30346, 29871,   232,
           135,   173, 31642, 29871, 31569, 31602, 29871, 31373, 31400, 29871,
           232,   164,   157, 30395, 29871, 30824,    13,   235,   180,   141,
         31392, 31615, 31438, 31030, 30415, 3

In [ ]:
print(summary)

あなたは教師です．以下の論文の内容を解釈し，生徒に説明してください．: # ニューラル機械翻訳における Iterative Back-Translation を利用した コンパラブルコーパスの活用

山本 優紀 秋葉 友良 塚田 元
豊橋技術科学大学
\{yamamoto.yuki.pr, akiba.tomoyoshi.tk, tsukada.hajime.hl\}@tut.jp

## 概要

ニューラル機械翻訳 (NMT) の学習に用いる対訳コーパスの構築法として, 文書単位で対応付けられた 2 つの言語のコーパス (コンパラブルコーパス) から、対応付けられる文ペアを自動的に抽出する手法が広く採用されている. しかし, 文単位で意味が対応するものは少なく,多くの文は抽出されず捨てられてしまう. 本研究では、対訳コーパスとして抽出されなかった文を含めて,コンパラブルコー パス全体を NMT の学習に活用する手法を提案する. 評価実験により, コンパラブルコーパスでデータ拡張を行うことや, コンパラブル性の利用, Iterative Back-Translation の活用によって, NMT の学習効果が向上することが確認された.

## 結果

本研究では、コンパラブルコーパス全体を NMT の学習に活用する手法を提案し、評価実験を行いました。コンパ
